In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE103737"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE103737"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE103737.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE103737.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE103737.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional correlates of high norepinephrine content in ovarian carcinomas"
!Series_summary	"Genome-wide transcriptome profiling was conducted on 97 Stage I-IV ovarian carcinomas classified according to tissue norepinephrine content."
!Series_overall_design	"To characterize the impact of norepinephrine on ovarian carcinoma biology, we conducted genome-wide transcriptome profiling of 97 clinical ovarian tumors.  Tissue norepinephrine content was assessed by high performance liquid chromatogrphy (0=below median value of 1.05 pg/ml;1=above median).  Covariates included age (years), body mass index (kg / m^2), tumor stage (1-4), and tumor grade (0=low;1=high)."
Sample Characteristics Dictionary:
{0: ['subject age: 70', 'subject age: 49', 'subject age: 87', 'subject age: 65', 'subject age: 63', 'subject age: 50', 'subject age: 71', 'subject age: 62', 'subject age: 51', 'subject age: 68', 'subject age: 48', 'subject age: 61', 'subject age: 79', 'su

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on series title and overall design, this appears to be gene expression data
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary:
# - For trait: norepinephrine content is in position 5
# - For age: subject age is in position 0
# - There is no gender information

trait_row = 5  # norepinephrine content
age_row = 0    # subject age
gender_row = None  # gender not available

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert norepinephrine content to binary values (0 or 1).
    Format: "norepinephrine content (0=below median value of 1.05 pg/ml;1=above median): 0/1"
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(':')
    if len(parts) < 2:
        return None
    
    value_str = parts[1].strip()
    try:
        # Convert to integer (already binary)
        return int(value_str)
    except:
        return None

def convert_age(value):
    """
    Convert age to continuous values.
    Format: "subject age: XX"
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(':')
    if len(parts) < 2:
        return None
    
    value_str = parts[1].strip()
    try:
        # Convert to float
        return float(value_str)
    except:
        return None

# Gender conversion function not needed as gender data is not available

# 3. Save Metadata
# Determine if trait data is available
is_trait_available = trait_row is not None
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Load the clinical data from the previous step
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.pkl")
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_pickle(clinical_data_path)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=None  # No gender data
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Save to CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['7A5', 'A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1',
       'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AACSL', 'AADAC',
       'AADACL1', 'AADACL2', 'AADACL3', 'AADACL4'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers based on the provided preview
# These appear to be proper human gene symbols. For example:
# A1BG (Alpha-1-B Glycoprotein)
# A2M (Alpha-2-Macroglobulin)
# AAAS (Aladin WD Repeat Nucleoporin)
# AACS (Acetoacetyl-CoA Synthetase)

# Some might be older gene symbols that have been updated, but they are gene symbols nonetheless,
# not probe IDs that would require mapping.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Re-extract clinical features since they weren't properly saved in a previous step
# Define the conversion functions again
def convert_trait(value):
    """
    Convert norepinephrine content to binary values (0 or 1).
    Format: "norepinephrine content (0=below median value of 1.05 pg/ml;1=above median): 0/1"
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(':')
    if len(parts) < 2:
        return None
    
    value_str = parts[1].strip()
    try:
        # Convert to integer (already binary)
        return int(value_str)
    except:
        return None

def convert_age(value):
    """
    Convert age to continuous values.
    Format: "subject age: XX"
    """
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    # Extract value after colon
    parts = value.split(':')
    if len(parts) < 2:
        return None
    
    value_str = parts[1].strip()
    try:
        # Convert to float
        return float(value_str)
    except:
        return None

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=5,  # norepinephrine content is in position 5
    convert_trait=convert_trait,
    age_row=0,    # subject age is in position 0
    convert_age=convert_age,
    gender_row=None,
    convert_gender=None
)

# Save clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Cohort contains ovarian cancer patients with norepinephrine content data."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Normalized gene data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE103737.csv
Clinical features saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE103737.csv


For the feature 'Ovarian_Cancer', the least common label is '0.0' with 46 occurrences. This represents 47.42% of the dataset.
The distribution of the feature 'Ovarian_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 50.0
  50% (Median): 60.0
  75%: 68.0
Min: 29.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Ovarian_Cancer/cohort_info.json


Linked data saved to ../../output/preprocess/Ovarian_Cancer/GSE103737.csv
